<a href="https://colab.research.google.com/github/chanyelee/Dacon-electricity-forecast/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- train.csv : 85일 간의 전력소비량(kWh)가 포함된 데이터
- test.csv : 85일 이후 7일 간의 전력소비량(kWh)을 예측해야하는 데이터
- building.info.csv : 10개 종류의 100개 건물의 데이터
- sample_submission.csv : test.csv로 예측한 전력소비량(kWh)을 데이콘에 제출하는 csv 파일

- "데이터_전처리.ipynb"에서 데이터 전처리 후 9개의 csv 파일을 추출 -> 추출한 csv 파일로 모델 학습 및 예측 수행

In [ ]:
import os
from google.colab import drive

drive.mount('/content/drive')

try:
    path = '/content/drive/MyDrive/위아이티'
    os.chdir(path)
    print(f"현재 작업 디렉터리: {os.getcwd()}")
except FileNotFoundError:
    print(f"오류: '{path}' 폴더를 찾을 수 없습니다. 경로를 확인해주세요.")

print("\n현재 폴더 내 파일 목록:")
!ls

Mounted at /content/drive
현재 작업 디렉터리: /content/drive/MyDrive/위아이티

현재 폴더 내 파일 목록:
building_info.csv      SAINT_6.ipynb	      train.csv
final_answer.csv       SAINT_7.ipynb	      trained_models
granite-tsfm	       saint_sample.csv       trained_models_final
hypergraph-tabular-lm  sample_1.csv	      train_group_1.csv
HyTrel		       sample_2.csv	      train_group_2.csv
__pycache__	       sample.csv	      train_group_3.csv
SAINT_1.ipynb	       sample_submission.csv  TST.ipynb
SAINT_2.ipynb	       test.csv		      valid_group_1.csv
SAINT_3.ipynb	       test_group_1.csv       valid_group_2.csv
SAINT_4.ipynb	       test_group_2.csv       valid_group_3.csv
SAINT_5.ipynb	       test_group_3.csv


In [ ]:
import numpy as np
import pandas as pd
import holidays
import math
import copy

In [ ]:
building = pd.read_csv('building_info.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

# ✅ 'hour' 컬럼 생성
train['hour'] = train['일시'].str[-2:].astype(int)
test['hour'] = test['일시'].str[-2:].astype(int)

# ✅ 건물번호 + 시간대별 평균 일조/일사 계산 후 test에 병합
illum_mean = (
    train.groupby(['건물번호', 'hour'])[['일조(hr)', '일사(MJ/m2)']]
    .mean()
    .reset_index()
)
test = test.merge(illum_mean, on=['건물번호', 'hour'], how='left')

In [ ]:
import pandas as pd
import numpy as np
import holidays

# 요일 구분 (평일=0, 토요일=1, 일요일/공휴일=2)
def get_day_type(dt):
    if dt.weekday() == 5: # 토요일
        return 1
    elif dt.weekday() == 6 or dt in kr_holidays: # 일요일 또는 공휴일
        return 2
    else: # 평일
        return 0

# 데이터프레임에 대한 피처 엔지니어링을 수행
def feature_engineering(df, kr_holidays):

    # 원본 데이터프레임 복사
    df_copy = df.copy()

    # '일시'를 datetime으로 변환
    df_copy['일시_dt'] = pd.to_datetime(df_copy['일시'], format='%Y%m%d %H')

    # 시간 관련 피처 생성
    df_copy['월'] = df_copy['일시_dt'].dt.month
    df_copy['요일'] = df_copy['일시_dt'].dt.weekday # 월요일=0, ..., 일요일=6
    df_copy['요일구분'] = df_copy['일시_dt'].apply(get_day_type)

    start_date = pd.to_datetime('2024-06-08')
    df_copy['Time_Index'] = (df_copy['일시_dt'].dt.normalize() - start_date).dt.days + 1

    # 시간 변수의 주기성을 반영하기 위한 sin/cos 변환
    period = 24
    # 'hour' 컬럼이 없는 경우 '일시_dt'에서 생성
    if 'hour' not in df_copy.columns:
        df_copy['hour'] = df_copy['일시_dt'].dt.hour

    df_copy['hour_sin'] = np.sin(2 * np.pi * df_copy['hour'] / period)
    df_copy['hour_cos'] = np.cos(2 * np.pi * df_copy['hour'] / period)

    # 파생 변수 생성
    # 1. 체감온도 (Discomfort Index)
    df_copy['체감온도'] = (
        df_copy['기온(°C)'] + 0.33 * df_copy['습도(%)'] - 0.70 * df_copy['풍속(m/s)'] - 4.00
    )

    # 2. 온습도지수 (THI: Temperature-Humidity Index)
    df_copy['온습도지수'] = (
        0.81 * df_copy['기온(°C)'] + 0.01 * df_copy['습도(%)'] * (0.99 * df_copy['기온(°C)'] - 14.3) + 46.3
    )

    # 3. 강수여부 (0 또는 1)
    df_copy['강수여부'] = df_copy['강수량(mm)'].apply(lambda x: 1 if x > 0 else 0)

    return df_copy

In [ ]:
all_years = pd.to_datetime(train['일시'], format='%Y%m%d %H').dt.year.unique().tolist()
kr_holidays = holidays.KR(years=all_years)

# 함수를 사용하여 train과 test 데이터에 피처 엔지니어링 적용
train_featured = feature_engineering(train, kr_holidays)
test_featured = feature_engineering(test, kr_holidays)

In [ ]:
train_featured['일시_dt_converted'] = pd.to_datetime(train_featured['일시_dt'], format='%Y%m%d %H')
test_featured['일시_dt_converted'] = pd.to_datetime(test_featured['일시_dt'], format='%Y%m%d %H')

# 1. 데이터 정렬
# 각 건물의 시간 순서가 보장되어야 정확한 Lag Feature를 만들 수 있습니다.
train_featured = train_featured.sort_values(by=['건물번호', '일시_dt_converted']).reset_index(drop=True)


# 2. 건물별로 그룹화하여 Lag Feature 생성
# shift() 함수는 N칸 만큼 데이터를 뒤로 밀어주는 역할을 합니다.
# groupby('건물번호')를 통해 각 건물의 시계열 내에서만 shift가 적용됩니다.
train_featured['1시간_전_전력소비량'] = train_featured.groupby('건물번호')['전력소비량(kWh)'].shift(1)
train_featured['하루_전_전력소비량'] = train_featured.groupby('건물번호')['전력소비량(kWh)'].shift(24)
train_featured['일주일_전_전력소비량'] = train_featured.groupby('건물번호')['전력소비량(kWh)'].shift(168)


# 3. 생성된 결측값(NaN)을 제거
# shift로 인해 생긴 빈 값(과거 데이터가 없는 초기 시점)을 제거합니다.
lag_features = ['1시간_전_전력소비량', '하루_전_전력소비량', '일주일_전_전력소비량']
train_featured = train_featured.dropna(subset=lag_features)

In [ ]:
# 2. test 데이터에 '일주일 전'에 해당하는 시간 키(key) 생성
# 각 test 데이터의 시간으로부터 정확히 7일 전의 시간을 계산하여 새 컬럼으로 만듭니다.
train_subset = train_featured[['건물번호', '일시_dt_converted', '전력소비량(kWh)']].rename(
    columns={'전력소비량(kWh)': '일주일_전_전력소비량'}
)
test_featured['lookup_timestamp'] = test_featured['일시_dt_converted'] - pd.Timedelta(days=7)

# 3. test 데이터와 과거 데이터를 merge하여 피처 추가
# test 데이터의 'lookup_timestamp'와 과거 데이터(train_subset)의 '일시_dt_converted'가
# 일치하는 것을 기준으로 두 데이터를 합칩니다.
# 'how='left''는 test_featured의 모든 행을 유지하라는 의미입니다.
test_featured = pd.merge(
    test_featured,
    train_subset,
    left_on=['건물번호', 'lookup_timestamp'],
    right_on=['건물번호', '일시_dt_converted'],
    how='left'
)

# 4. 불필요한 컬럼 정리
# merge에 사용했던 조회용 컬럼들을 삭제하여 데이터프레임을 깔끔하게 정리합니다.
test_featured = test_featured.drop(columns=['lookup_timestamp', '일시_dt_converted_y'])
# 컬럼 이름이 겹치는 경우 _x, _y가 붙을 수 있으므로, 원본 컬럼 이름으로 다시 정리합니다.
test_featured = test_featured.rename(columns={'일시_dt_converted_x': '일시_dt_converted'})

In [ ]:
# 1. 경계 데이터 연결
# train_featured의 마지막 날짜(8월 24일) 데이터만 선택합니다.
last_day_of_train = train_featured[train_featured['일시_dt_converted'].dt.date == pd.to_datetime('2024-08-24').date()].copy()

# train 마지막 날짜와 test 데이터 전체를 합칩니다.
# 이렇게 하면 8월 24일 -> 8월 25일로 데이터가 끊김 없이 이어집니다.
combined_df = pd.concat([last_day_of_train, test_featured], ignore_index=True)

# 건물번호와 시간순으로 데이터를 정렬하여 정확한 순서를 보장합니다.
combined_df = combined_df.sort_values(by=['건물번호', '일시_dt_converted']).reset_index(drop=True)


# 2. 연결된 데이터프레임에서 Lag Feature 일괄 생성
# groupby와 shift를 사용하면, test(8월 25일)의 Lag Feature가
# train(8월 24일)의 값을 참조하여 정확하게 생성됩니다.
combined_df['1시간_전_전력소비량'] = combined_df.groupby('건물번호')['전력소비량(kWh)'].shift(1)
combined_df['하루_전_전력소비량'] = combined_df.groupby('건물번호')['전력소비량(kWh)'].shift(24)


# 3. 최종 Test 데이터만 다시 추출
# Lag Feature가 모두 채워졌으므로, 원래의 test 기간에 해당하는 데이터만 다시 선택합니다.
test_start_date = test_featured['일시_dt_converted'].min()
test_featured_final = combined_df[combined_df['일시_dt_converted'] >= test_start_date].copy()


# 4. 결과 확인
# 8월 25일 00시 데이터에 8월 24일의 값들이 잘 들어갔는지 확인합니다.
print("--- Lag Feature 추가 후 최종 test 데이터 (상위 5개) ---")
print(test_featured_final[['건물번호', '일시_dt_converted', '전력소비량(kWh)', '1시간_전_전력소비량', '하루_전_전력소비량']].head())

# 8월 25일 00시의 '1시간_전_전력소비량'이 8월 24일 23시의 값(9999)인지 확인
first_row = test_featured_final.iloc[0]
print(f"\n8월 25일 00시의 '1시간 전 전력소비량'은 {first_row['1시간_전_전력소비량']} 입니다.")
print("(이는 8월 24일 23시의 실제 전력소비량 값과 일치해야 합니다.)")

--- Lag Feature 추가 후 최종 test 데이터 (상위 5개) ---
    건물번호     일시_dt_converted  전력소비량(kWh)  1시간_전_전력소비량  하루_전_전력소비량
24     1 2024-08-25 00:00:00         NaN      4499.01     4897.59
25     1 2024-08-25 01:00:00         NaN          NaN     4664.22
26     1 2024-08-25 02:00:00         NaN          NaN     3716.73
27     1 2024-08-25 03:00:00         NaN          NaN     3073.47
28     1 2024-08-25 04:00:00         NaN          NaN     3399.63

8월 25일 00시의 '1시간 전 전력소비량'은 4499.01 입니다.
(이는 8월 24일 23시의 실제 전력소비량 값과 일치해야 합니다.)


In [ ]:
train_featured['전력소비량(kWh)_log'] = np.log1p(train_featured['전력소비량(kWh)'])

date_to_remove = pd.to_datetime('2024-06-08')

# 2. 조건을 만족하는 데이터만 선택하여 덮어쓰기
train_featured = train_featured[
    (train_featured['일시_dt_converted'] >= date_to_remove)
]

# 2. 검증 기간 설정
validation_start_date = '2024-08-18'

# 3. 새로 만든 datetime 컬럼을 기준으로 훈련 세트와 검증 세트 분할
model_train_set = train_featured[train_featured['일시_dt_converted'] < validation_start_date].reset_index(drop=True)
model_valid_set = train_featured[train_featured['일시_dt_converted'] >= validation_start_date].reset_index(drop=True)

In [ ]:
def process_building_features(df):
    """
    건물 설비 정보(태양광, ESS, PCS)를 바탕으로 '장비그룹'을 생성하고,
    '건물유형'을 숫자형으로 변환하는 함수입니다.

    """
    # 원본 수정을 방지하기 위해 데이터프레임 복사
    df_copy = df.copy()

    # --- 장비그룹 생성 ---
    # 1. 각 장비 보유 여부를 True(1)/False(0)로 변환
    # '-'가 아니면 장비가 있는 것(True)으로 간주
    has_태양광 = (df_copy["태양광용량(kW)"] != '-').astype(int)
    has_ESS = (df_copy["ESS저장용량(kWh)"] != '-').astype(int)
    has_PCS = (df_copy["PCS용량(kW)"] != '-').astype(int)

    # 2. 보유 장비 수 계산
    total_equip = has_태양광 + has_ESS + has_PCS

    # 3. 조건에 따라 그룹 분류 (np.select 사용으로 가독성 및 성능 개선)
    # 원본 코드에서는 장비가 2개인 경우를 다루지 않았으나,
    # 여기서는 '일부만 있음'으로 간주하여 그룹 2에 포함시키는 더 안정적인 로직을 사용합니다.
    conditions = [
        total_equip == 0,  # 장비가 하나도 없으면 그룹 1
        total_equip == 3   # 장비를 모두 보유하면 그룹 3
    ]
    choices = [1, 3]
    df_copy['장비그룹'] = np.select(conditions, choices, default=2) # 그 외(1개 또는 2개 보유)는 그룹 2

    # --- 건물유형 인코딩 ---
    # '건물유형' 컬럼이 존재하면 카테고리 코드로 변환
    if '건물유형' in df_copy.columns:
        df_copy['건물유형'] = df_copy['건물유형'].astype('category').cat.codes

    return df_copy

In [ ]:
def process_and_group_data(main_df, building_df):
    """
    메인 데이터에 건물 정보를 병합하고, '장비그룹'에 따라 3개로 분리합니다.

    """
    # 1. building 데이터 처리 및 병합
    building_processed = process_building_features(building_df)
    merged_df = pd.merge(main_df, building_processed, on='건물번호', how='left')

    # 2. '장비그룹'을 기준으로 데이터프레임 분리
    # .copy()를 사용하여 SettingWithCopyWarning 방지
    group_1 = merged_df[merged_df['장비그룹'] == 1].copy()
    group_2 = merged_df[merged_df['장비그룹'] == 2].copy()
    group_3 = merged_df[merged_df['장비그룹'] == 3].copy()

    # 3. 3개의 데이터프레임 반환
    return group_1, group_2, group_3

In [ ]:
train_group_1, train_group_2, train_group_3 = process_and_group_data(model_train_set, building)
valid_group_1, valid_group_2, valid_group_3 = process_and_group_data(model_valid_set, building)
test_group_1, test_group_2, test_group_3 = process_and_group_data(test_featured_final, building)

In [ ]:
def optimize_memory(df: pd.DataFrame) -> pd.DataFrame:
    """
    데이터프레임의 int64, float64 타입을 각각 int32, float32로 변경하여 메모리를 최적화합니다.
    """
    print("========== 메모리 최적화 시작 ==========")

    # 변경 전 메모리 사용량 확인
    mem_before = df.memory_usage(deep=True).sum()
    print(f"--- 변경 전 메모리: {mem_before / 1e6:.2f} MB ---")

    # 변경할 타입을 담을 딕셔너리 생성
    conversion_dict = {}
    for col, dtype in df.dtypes.items():
        if dtype == 'int64':
            conversion_dict[col] = 'int32'
        elif dtype == 'float64':
            conversion_dict[col] = 'float32'

    # astype을 이용해 한 번에 변경
    if conversion_dict:
        df = df.astype(conversion_dict)
        mem_after = df.memory_usage(deep=True).sum()

        print(f"--- 변경 후 메모리: {mem_after / 1e6:.2f} MB ---")

        # 메모리 절감량 요약
        reduction = ((mem_before - mem_after) / mem_before) * 100
        print(f"✅ 총 {reduction:.2f}%의 메모리를 절감했습니다.")
    else:
        print("✅ 변경할 int64 또는 float64 타입의 컬럼이 없습니다.")

    print("========== 메모리 최적화 완료 ==========\n")
    return df

In [ ]:
TARGET_COL = ['전력소비량(kWh)']
TARGET_COL_LOG = ['전력소비량(kWh)_log']

CAT_COLS = ['건물번호', 'hour', '요일구분', '월', '요일', '건물유형', '강수여부', 'num_date_time']

g1_cont_cols = [
    '기온(°C)', '강수량(mm)', '풍속(m/s)', '습도(%)', 'hour_sin', 'hour_cos', 'Time_Index',
    '연면적(m2)', '냉방면적(m2)', '일조(hr)', '일사(MJ/m2)', '체감온도', '온습도지수', '일주일_전_전력소비량',
    '1시간_전_전력소비량', '하루_전_전력소비량',
]

g2_cont_cols = g1_cont_cols + ['태양광용량(kW)']
g3_cont_cols = g2_cont_cols + ['ESS저장용량(kWh)', 'PCS용량(kW)']

In [ ]:
train_1_optimized_df = optimize_memory(train_group_1[TARGET_COL + TARGET_COL_LOG + CAT_COLS + g1_cont_cols])
train_2_optimized_df = optimize_memory(train_group_2[TARGET_COL + TARGET_COL_LOG + CAT_COLS + g2_cont_cols])
train_3_optimized_df = optimize_memory(train_group_3[TARGET_COL + TARGET_COL_LOG + CAT_COLS + g3_cont_cols])

========== 메모리 최적화 시작 ==========
--- 변경 전 메모리: 21.14 MB ---
--- 변경 후 메모리: 14.09 MB ---
✅ 총 33.34%의 메모리를 절감했습니다.
========== 메모리 최적화 완료 ==========

========== 메모리 최적화 시작 ==========
--- 변경 전 메모리: 22.76 MB ---
--- 변경 후 메모리: 16.61 MB ---
✅ 총 27.02%의 메모리를 절감했습니다.
========== 메모리 최적화 완료 ==========

========== 메모리 최적화 시작 ==========
--- 변경 전 메모리: 9.16 MB ---
--- 변경 후 메모리: 7.36 MB ---
✅ 총 19.65%의 메모리를 절감했습니다.
========== 메모리 최적화 완료 ==========



In [ ]:
valid_1_optimized_df = optimize_memory(valid_group_1[TARGET_COL + TARGET_COL_LOG + CAT_COLS + g1_cont_cols])
valid_2_optimized_df = optimize_memory(valid_group_2[TARGET_COL + TARGET_COL_LOG + CAT_COLS + g2_cont_cols])
valid_3_optimized_df = optimize_memory(valid_group_3[TARGET_COL + TARGET_COL_LOG + CAT_COLS + g3_cont_cols])

========== 메모리 최적화 시작 ==========
--- 변경 전 메모리: 2.08 MB ---
--- 변경 후 메모리: 1.39 MB ---
✅ 총 33.34%의 메모리를 절감했습니다.
========== 메모리 최적화 완료 ==========

========== 메모리 최적화 시작 ==========
--- 변경 전 메모리: 2.24 MB ---
--- 변경 후 메모리: 1.64 MB ---
✅ 총 27.02%의 메모리를 절감했습니다.
========== 메모리 최적화 완료 ==========

========== 메모리 최적화 시작 ==========
--- 변경 전 메모리: 0.90 MB ---
--- 변경 후 메모리: 0.73 MB ---
✅ 총 19.65%의 메모리를 절감했습니다.
========== 메모리 최적화 완료 ==========



In [ ]:
test_1_optimized_df = optimize_memory(test_group_1[CAT_COLS + g1_cont_cols])
test_2_optimized_df = optimize_memory(test_group_2[CAT_COLS + g2_cont_cols])
test_3_optimized_df = optimize_memory(test_group_3[CAT_COLS + g3_cont_cols])

========== 메모리 최적화 시작 ==========
--- 변경 전 메모리: 1.96 MB ---
--- 변경 후 메모리: 1.33 MB ---
✅ 총 32.27%의 메모리를 절감했습니다.
========== 메모리 최적화 완료 ==========

========== 메모리 최적화 시작 ==========
--- 변경 전 메모리: 2.13 MB ---
--- 변경 후 메모리: 1.58 MB ---
✅ 총 25.83%의 메모리를 절감했습니다.
========== 메모리 최적화 완료 ==========

========== 메모리 최적화 시작 ==========
--- 변경 전 메모리: 0.87 MB ---
--- 변경 후 메모리: 0.71 MB ---
✅ 총 18.52%의 메모리를 절감했습니다.
========== 메모리 최적화 완료 ==========



In [ ]:
train_1_optimized_df.to_csv('train_group_1.csv', index=False)
train_2_optimized_df.to_csv('train_group_2.csv', index=False)
train_3_optimized_df.to_csv('train_group_3.csv', index=False)

valid_1_optimized_df.to_csv('valid_group_1.csv', index=False)
valid_2_optimized_df.to_csv('valid_group_2.csv', index=False)
valid_3_optimized_df.to_csv('valid_group_3.csv', index=False)

test_1_optimized_df.to_csv('test_group_1.csv', index=False)
test_2_optimized_df.to_csv('test_group_2.csv', index=False)
test_3_optimized_df.to_csv('test_group_3.csv', index=False)